# 13-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.

- mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
- ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

프로젝트에 필요한 주요 라이브러리 버전을 확인해봅시다.

In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

In [7]:
# 영화를 title로 입력할 것이기때문에 title이란 컬럼을 불러와야합니다.
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


### 2) 분석해 봅시다.


- ratings에 있는 유니크한 영화 개수

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

- ratings에 있는 유니크한 사용자 수

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

- 가장 인기 있는 영화 30개(인기순)

In [11]:
#가장 인기 있는 영화 30개(인기순) 영화 id별 별점(시청횟수)많은 순
movie_count = ratings.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [12]:
# 내가 선호하는 영화를 5가지 골라서 rating에 추가
my_favorite = ['minority report', 'mission impossible', 'get out' , 'inception', 'gravity']

my_list = pd.DataFrame({'user_id': ['meissamatar']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['meissamatar']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)       

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,meissamatar,NaN,5,NaN,minority report
1,meissamatar,NaN,5,NaN,mission impossible
2,meissamatar,NaN,5,NaN,get out
3,meissamatar,NaN,5,NaN,inception
4,meissamatar,NaN,5,NaN,gravity


In [13]:
# 인덱싱
userid_unique = ratings['user_id'].unique()
#movie_unique = ratings['movie_id'].unique()

movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['meissamatar'])    
print(movie_to_idx['inception'])

6039
3631


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,minority report
1,6039,3629,5,NaN,mission impossible
2,6039,3630,5,NaN,get out
3,6039,3631,5,NaN,inception


### 4) CSR matrix를 직접 만들어 봅시다.

In [17]:
# csr_matrix
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다.
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
meissamatar, inception = user_to_idx['meissamatar'], movie_to_idx['inception']
meissamatar_vector, inception_vector = als_model.user_factors[meissamatar], als_model.item_factors[inception]

print('슝=3')

슝=3


In [22]:
# 내적하는 코드
np.dot(meissamatar_vector, inception_vector)

0.0015335141

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [23]:
getout = movie_to_idx['get out']
getout_vector = als_model.item_factors[getout]
np.dot(meissamatar_vector, getout_vector)

0.0013636093

In [24]:
# (id, 유사도 )
favorite_movie = 'inception'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 1.0000001),
 (3632, 0.99768496),
 (3628, 0.9965461),
 (3629, 0.9933596),
 (3630, 0.989488),
 (3578, 0.98588663),
 (3579, 0.98569715),
 (3575, 0.9851869),
 (3574, 0.9848779),
 (3576, 0.9835451),
 (3577, 0.9834415),
 (3583, 0.98296624),
 (3573, 0.98250526),
 (3572, 0.9821173),
 (3570, 0.9815562)]

In [25]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['inception',
 'gravity',
 'minority report',
 'mission impossible',
 'get out',
 'Number Seventeen (1932)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "Brother's Kiss, A (1997)",
 'Century (1993)',
 'War at Home, The (1996)',
 'Daens (1992)',
 'Neon Bible, The (1995)',
 'Male and Female (1919)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Loser (1991)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [26]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [27]:
get_similar_movie('get out')

['get out',
 'mission impossible',
 'minority report',
 'gravity',
 'inception',
 'Number Seventeen (1932)',
 "Brother's Kiss, A (1997)",
 'Century (1993)',
 'War at Home, The (1996)',
 'Neon Bible, The (1995)']

In [28]:
user = user_to_idx['meissamatar']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(762, 0.010386819),
 (2894, 0.009255593),
 (170, 0.009106078),
 (232, 0.009045634),
 (3000, 0.008859548),
 (2229, 0.008826815),
 (2726, 0.008509925),
 (243, 0.008502917),
 (126, 0.008395117),
 (241, 0.008372763),
 (2977, 0.008354016),
 (3309, 0.008326583),
 (832, 0.00826573),
 (2593, 0.008178965),
 (1854, 0.008153323),
 (1412, 0.00814372),
 (2692, 0.008113563),
 (669, 0.008110933),
 (2430, 0.008051554),
 (2129, 0.008002499)]

In [29]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Great Escape, The (1963)',
 'When Night Is Falling (1995)',
 'Being John Malkovich (1999)',
 'Beautiful Thing (1996)',
 'Female Perversions (1996)',
 'Thieves (Voleurs, Les) (1996)',
 'Daytrippers, The (1996)',
 'Ghostbusters (1984)',
 'Shakespeare in Love (1998)',
 'Pillow Book, The (1995)',
 'Bitter Moon (1992)',
 'Hollow Reed (1996)',
 'Jakob the Liar (1999)',
 'My Favorite Season (1993)',
 'Before and After (1996)',
 'Right Stuff, The (1983)',
 'Amateur (1994)',
 'Man on the Moon (1999)',
 'Sanjuro (1962)',
 'Character (Karakter) (1997)']

## 프로젝트 결과 및 회고
- 이번 프로젝트는 추천 시스템에 대한 프로젝트 입니다. 
- CSR Matrix를 만들고 AlternatingLeastSquares 모델을 구성하여 학습시킨 후 내적을 통해 선호도를 파악하고 입력한 영화와 비슷한 영화를 추천받고 좋아할 만한 영화들을 추천받을 수 있도록 구현해 보았습니다. 
- 결과를 살펴보면 어느 정도 취향과 비슷한 영화를 추천해준 것을 유사도를 통해 확인할 수 있었으며 추후에 AlternatingLeastSquares 모델의 파라미터를 바꾸어가며 결과의 변화를 지켜볼 예정입니다.